<a href="https://colab.research.google.com/github/samarak/coursera_capstone/blob/master/3_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segmenting and Clustering Neighborhoods in Toront part 3
## we work with only boroughs that contain the word Toronto 

In [0]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [0]:
#load df
df=pd.read_csv('df2.csv')
df.drop(df.columns[0], axis=1, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


##### let's see all the Boroughs with Toronto in their name 

In [0]:
Tdf = df [df['Borough'].str.contains('Toronto|toronto',regex=True)].reset_index(drop=True)
print('The dataframe has {} boroughs '.format(
        len(Tdf['Borough'].unique())
    )
)
Tdf.head()

The dataframe has 4 boroughs 


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


##### lets plot on the folium all these spots

In [0]:
latt=43.6532
lont=-79.3832
map_toronto = folium.Map(location=[latt, lont], zoom_start=12)

for lat, lng, borough, neighborhood,postcode in zip(Tdf['Latitude'], Tdf['Longitude'], Tdf['Borough'], Tdf['Neighbourhood'], Tdf['Postcode']):
    label = '{}, {}, {}'.format(neighborhood, borough,postcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them. 
##### lets connect to the foursquare 

In [0]:
CLIENT_ID = 'FT2BY0TIEZTRWT5EZQHKZ5I5OBXQRN4VB5PMDRFMZ3HKXDCT' # your Foursquare ID
CLIENT_SECRET = 'B0P15OIEILAR4MPRFHLXMI0FFU5MD53EWTKRJK0MEWRBYAO5' # your Foursquare Secret
VERSION = '20181204'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FT2BY0TIEZTRWT5EZQHKZ5I5OBXQRN4VB5PMDRFMZ3HKXDCT
CLIENT_SECRET:B0P15OIEILAR4MPRFHLXMI0FFU5MD53EWTKRJK0MEWRBYAO5


In [0]:
# lets explore the first Borough in df
newdf = Tdf.loc[0,['Borough','Latitude', 'Longitude']]
Borough ,       Latitude ,     Longitude     =          newdf.values

In [0]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import requests

# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Latitude, 
    Longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=FT2BY0TIEZTRWT5EZQHKZ5I5OBXQRN4VB5PMDRFMZ3HKXDCT&client_secret=B0P15OIEILAR4MPRFHLXMI0FFU5MD53EWTKRJK0MEWRBYAO5&v=20181204&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [0]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c8045ad1ed2196e4aaba3fd'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c062f964a52011f820e3',
       'name': 'The Big Carrot Natural Food Market',
       'location': {'address': '125 Southwood Dr',
        'lat': 43.678879,
        'lng': -79.297734,
        'labeledLatLngs': [{'label': 'display',
          'lat

In [0]:
#convert to data frame
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


,name,categories,lat,lng
0,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
1,Grover Pub and Grub,Pub,43.679181,-79.297215
2,Starbucks,Coffee Shop,43.678798,-79.298045
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [0]:
#extract all 
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Toronto_venues = getNearbyVenues(names=Tdf['Borough'],
                                   latitudes=Tdf['Latitude'],
                                   longitudes=Tdf['Longitude']
                                  )


East Toronto
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
West Toronto
West Toronto
West Toronto
West Toronto
West Toronto
West Toronto
East Toronto


In [0]:
Tdf['Borough'].unique()

array(['East Toronto', 'Central Toronto', 'Downtown Toronto',
       'West Toronto'], dtype=object)

In [0]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1692, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,East Toronto,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,East Toronto,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,East Toronto,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,East Toronto,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,East Toronto,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


Let's check how many venues were returned for each Borough

In [0]:
Toronto_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Central Toronto,113,113,113,113,113,113
Downtown Toronto,1277,1277,1277,1277,1277,1277
East Toronto,124,124,124,124,124,124
West Toronto,178,178,178,178,178,178


#### Let's find out how many unique categories can be curated from all the returned venues

In [0]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 234 uniques categories.


In [0]:
# one hot encoding
toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add Borough column back to dataframe
toronto_onehot['Borough'] = Toronto_venues['Borough'] 

# move Borough column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [0]:
toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()
toronto_grouped.head()

,Borough,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017699,...,0.008850,0.000000,0.008850,0.000000,0.008850,0.000000,0.000000,0.000000,0.000000,0.008850
1,Downtown Toronto,0.000783,0.000783,0.000783,0.000783,0.000783,0.001566,0.001566,0.001566,0.016445,...,0.000783,0.003915,0.008614,0.001566,0.003915,0.006265,0.000783,0.000783,0.001566,0.002349
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024194,...,0.008065,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024194
3,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005618,...,0.000000,0.000000,0.005618,0.000000,0.011236,0.005618,0.000000,0.000000,0.000000,0.005618


In [0]:
print(toronto_grouped.shape)

(4, 235)


#### Let's print each Borough along with the top 5 most common venues

In [0]:
num_top_venues = 5

for hood in toronto_grouped['Borough']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Central Toronto----
            venue  freq
0     Coffee Shop  0.08
1  Sandwich Place  0.07
2     Pizza Place  0.05
3    Dessert Shop  0.04
4            Park  0.04


----Downtown Toronto----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.06
2       Bakery  0.03
3   Restaurant  0.03
4        Hotel  0.03


----East Toronto----
                venue  freq
0    Greek Restaurant  0.08
1         Coffee Shop  0.07
2      Ice Cream Shop  0.04
3  Italian Restaurant  0.04
4                Café  0.03


----West Toronto----
                venue  freq
0                 Bar  0.07
1                Café  0.06
2         Coffee Shop  0.05
3         Pizza Place  0.03
4  Italian Restaurant  0.03




#### Let's put that into a *pandas* dataframe

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Borough'] = toronto_grouped['Borough']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Pizza Place,Café,Park,Dessert Shop,Sushi Restaurant,Pharmacy,Burger Joint,Pub
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,Pizza Place,American Restaurant
2,East Toronto,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Café,Brewery,Park,Pub,Bookstore,Bakery
3,West Toronto,Bar,Café,Coffee Shop,Bakery,Pizza Place,Italian Restaurant,Restaurant,Breakfast Spot,Diner,Park


# Run *k*-means to cluster the Borough into 2 clusters

In [0]:
# set number of clusters
kclusters = 2

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe


toronto_grouped['labels'] = kmeans.labels_
toronto_grouped

,Borough,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,labels
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017699,...,0.000000,0.008850,0.000000,0.008850,0.000000,0.000000,0.000000,0.000000,0.008850,0
1,Downtown Toronto,0.000783,0.000783,0.000783,0.000783,0.000783,0.001566,0.001566,0.001566,0.016445,...,0.003915,0.008614,0.001566,0.003915,0.006265,0.000783,0.000783,0.001566,0.002349,1
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024194,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024194,1
3,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005618,...,0.000000,0.005618,0.000000,0.011236,0.005618,0.000000,0.000000,0.000000,0.005618,1


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [0]:
#get location of Borough
Borough_coor = Toronto_venues.drop_duplicates('Borough').reset_index(drop=True)[['Borough','Borough Latitude', 'Borough Longitude']].set_index('Borough')
Borough_coor

toronto_venues_sorted['labels'] = kmeans.labels_
toronto_venues_sorted[['Borough Latitude', 'Borough Longitude']] = Borough_coor.loc[toronto_venues_sorted['Borough']].reset_index()[['Borough Latitude', 'Borough Longitude']]
toronto_venues_sorted


,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,labels,Borough Latitude,Borough Longitude
0,Central Toronto,Coffee Shop,Sandwich Place,Pizza Place,Café,Park,Dessert Shop,Sushi Restaurant,Pharmacy,Burger Joint,Pub,0,43.728020,-79.388790
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Bar,Japanese Restaurant,Pizza Place,American Restaurant,1,43.679563,-79.377529
2,East Toronto,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Café,Brewery,Park,Pub,Bookstore,Bakery,1,43.676357,-79.293031
3,West Toronto,Bar,Café,Coffee Shop,Bakery,Pizza Place,Italian Restaurant,Restaurant,Breakfast Spot,Diner,Park,1,43.669005,-79.442259


In [0]:
# create map
map_clusters = folium.Map(location=[latt , lont], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_venues_sorted['Borough Latitude'], toronto_venues_sorted['Borough Longitude'], toronto_venues_sorted['Borough'], toronto_venues_sorted['labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

In [0]:
#cluster 1 
toronto_venues_sorted.loc[toronto_venues_sorted['labels'] == 0, toronto_venues_sorted.columns[[1] + list(range(5, toronto_venues_sorted.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,labels,Borough Latitude,Borough Longitude
0,Coffee Shop,Park,Dessert Shop,Sushi Restaurant,Pharmacy,Burger Joint,Pub,0,43.72802,-79.38879


In [0]:
#cluster 2
toronto_venues_sorted.loc[toronto_venues_sorted['labels'] == 1, toronto_venues_sorted.columns[[1] + list(range(5, toronto_venues_sorted.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,labels,Borough Latitude,Borough Longitude
1,Coffee Shop,Bakery,Italian Restaurant,Bar,Japanese Restaurant,Pizza Place,American Restaurant,1,43.679563,-79.377529
2,Greek Restaurant,Café,Brewery,Park,Pub,Bookstore,Bakery,1,43.676357,-79.293031
3,Bar,Pizza Place,Italian Restaurant,Restaurant,Breakfast Spot,Diner,Park,1,43.669005,-79.442259
